In [47]:
# Imports

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score


import duckdb

In [2]:
#Enforce copy on write
pd.options.mode.copy_on_write = True

Data far too large to upload - download from https://www.cdc.gov/brfss/annual_data/annual_2014.html (SAS format)


Ensure file name is LLCP2014.XPT (i had issues with file saving with extra space at end)

In [3]:
cwd = Path().cwd()
project_folder = cwd.parent
data_path = Path("data/LLCP2014.XPT")
file = project_folder / data_path

if not file.exists():
    raise FileNotFoundError("Data files not found. Please ensure the data files are in the correct directory.")

df = pd.read_sas(file)

print(df.head())

df.shape


   _STATE  FMONTH        IDATE IMONTH   IDAY    IYEAR  DISPCODE         SEQNO  \
0     1.0     1.0  b'01172014'  b'01'  b'17'  b'2014'    1100.0  2.014000e+09   
1     1.0     1.0  b'01072014'  b'01'  b'07'  b'2014'    1100.0  2.014000e+09   
2     1.0     1.0  b'01092014'  b'01'  b'09'  b'2014'    1100.0  2.014000e+09   
3     1.0     1.0  b'01072014'  b'01'  b'07'  b'2014'    1100.0  2.014000e+09   
4     1.0     1.0  b'01162014'  b'01'  b'16'  b'2014'    1100.0  2.014000e+09   

           _PSU  CTELENUM  ...  _FOBTFS  _CRCREC  _AIDTST3  _IMPEDUC  \
0  2.014000e+09       1.0  ...      2.0      1.0       2.0       5.0   
1  2.014000e+09       1.0  ...      2.0      2.0       2.0       4.0   
2  2.014000e+09       1.0  ...      2.0      2.0       2.0       6.0   
3  2.014000e+09       1.0  ...      2.0      1.0       2.0       6.0   
4  2.014000e+09       1.0  ...      2.0      1.0       2.0       5.0   

   _IMPMRTL  _IMPHOME  RCSBRAC1  RCSRACE1  RCHISLA1  RCSBIRTH  
0       1.0     

(464664, 279)

In [4]:
conn = duckdb.connect("brffs.duckdb")

# Register DataFrame
conn.register("df_view", df)

# Create table from df
conn.execute("""
    CREATE TABLE IF NOT EXISTS llcp2014 AS
    SELECT * FROM df_view
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014").fetchone())


(464664,)


Below I define variables deemed relevant to the data analysis. 

In [5]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS llcp2014_reduced AS
    SELECT 
        SLEPTIM1,
        SEX,
        _AGEG5YR,
        EDUCA,
        INCOME2,
        _RACEGR3,
        _HISPANC,
        _STATE,
        _BMI5,
        _BMI5CAT,
        _TOTINDA,
        _SMOKER3,
        _RFSMOK3,
        DRNK3GE5,
        _RFBING5,
        GENHLTH,
        MENTHLTH,
        CVDINFR4,
        CVDSTRK3,
        CHCCOPD1,
        ASTHMA3,
        DIABETE3,
        ADDEPEV2
    FROM llcp2014
""")


# Data Cleaning

### Invalidate values outside possible range, create a short sleep indicator from datapoints with sleep < 7 hours

In [6]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS sleep_hours INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET sleep_hours = 
        CASE 
            WHEN SLEPTIM1 IN (77, 99) THEN NULL
            WHEN SLEPTIM1 < 1 OR SLEPTIM1 > 24 THEN NULL
            ELSE CAST(SLEPTIM1 AS INTEGER)
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())

(458172,)


In [7]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS short_sleep INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET short_sleep =
        CASE
            WHEN sleep_hours IS NULL THEN NULL
            WHEN sleep_hours < 7 THEN 1
            ELSE 0
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


Adjust BMI from _BMI5, and account for missing values. 

In [8]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS bmi INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET bmi = 
        CASE 
            WHEN _BMI5 >= 9999 THEN NULL
            ELSE CAST(_BMI5 AS INTEGER)/100.0
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


### Assign sex to a binary value. 

### Assign education values to one of 3 values to make it ordinal. 

### Create binary values for low income, race (white) and race (hispanic)

In [9]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS sex_male INTEGER
""")

conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS educ_3cat INTEGER
""")

conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS low_income INTEGER
""")

conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS race_white_nh INTEGER
""")

conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS race_hispanic INTEGER
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [10]:
conn.execute("""
    UPDATE llcp2014_reduced
    SET sex_male = 
        CASE 
            WHEN SEX IN (7,9) THEN NULL
            WHEN SEX = 1 THEN 1
            WHEN SEX = 2 THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [11]:
conn.execute("""
    UPDATE llcp2014_reduced
    SET educ_3cat = 
        CASE 
            WHEN EDUCA = 9 THEN NULL
            WHEN EDUCA IN (1,2,3) THEN 0
            WHEN EDUCA IN (4,5) THEN 1
            WHEN EDUCA = 6 THEN 2
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [12]:
conn.execute("""
    UPDATE llcp2014_reduced
    SET low_income = 
        CASE 
            WHEN INCOME2 IN (77, 99) THEN NULL
            WHEN INCOME2 IN (1,2,3,4) THEN 1
            WHEN INCOME2 IN (5,6,7,8) THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [13]:
conn.execute("""
    UPDATE llcp2014_reduced
    SET race_white_nh = 
        CASE 
            WHEN _RACEGR3 = 9 THEN NULL
            WHEN _RACEGR3 = 1 THEN 1
            WHEN _RACEGR3 IN (2, 3, 4, 5) THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [14]:
conn.execute("""
    UPDATE llcp2014_reduced
    SET race_hispanic = 
        CASE 
            WHEN _RACEGR3 = 9 THEN NULL
            WHEN _RACEGR3 = 5 THEN 1
            WHEN _RACEGR3 IN (1, 2, 3, 4) THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


### take leisure physical activity and assign binary values for two different designations. 

In [15]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS any_leisure_pa INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET any_leisure_pa = 
        CASE 
            WHEN _TOTINDA = 9 THEN NULL
            WHEN _TOTINDA = 1 THEN 1
            WHEN _TOTINDA = 2 THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


### Adjust smoker categories to two binary categories. Smoker ever for individuals that have smoked before and smoker current for individuals that currently smoke. 

In [16]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS smoker_current INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET smoker_current = 
        CASE 
            WHEN _SMOKER3 = 9 THEN NULL
            ELSE CAST(_SMOKER3 AS INTEGER)
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [17]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS smoker_ever DOUBLE
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET smoker_ever = 
        CASE 
            WHEN _SMOKER3 = 9 THEN NULL
            WHEN _SMOKER3 IN (1,2,3) THEN 1
            WHEN _SMOKER3 = 4 THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [18]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS binge_drink INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET binge_drink = 
        CASE 
            WHEN _RFBING5 = 9 THEN NULL
            WHEN _RFBING5 = 2 THEN 1
            WHEN _RFBING5 = 1 THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [19]:
conn.execute("""
    UPDATE llcp2014_reduced
    SET smoker_current = 
        CASE 
            WHEN _SMOKER3 = 9 THEN NULL
            ELSE CAST(_SMOKER3 AS INTEGER)
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


### assign the general health rating to one of two categories, good health and poor health. 

### assign mental_unhealthy days to valid values of METNTHLTH column, and assign a binary value for poor mental health days being frequent (>=14) or infrequent (<14)

In [20]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS fairpoor_health INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET fairpoor_health =
        CASE
            WHEN GENHLTH IN (4, 5) THEN 1
            WHEN GENHLTH IN (1, 2, 3) THEN 0
            WHEN GENHLTH IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [21]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS mental_unhealthy_days FLOAT
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET mental_unhealthy_days =
        CASE
            WHEN MENTHLTH IN (77, 99) THEN NULL
            WHEN MENTHLTH = 88 THEN 0
            ELSE CAST(MENTHLTH AS FLOAT)
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [22]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS frequent_mental_distress INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET frequent_mental_distress =
        CASE
            WHEN mental_unhealthy_days >= 14 THEN 1
            WHEN mental_unhealthy_days < 14 THEN 0
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


### take all of the diagnosis variables and store them as 0 for never having had, and 1 for having had. 

### Apply the same logic to different diabetes categories. 

In [23]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS cvdinfr4_ever INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET cvdinfr4_ever =
        CASE
            WHEN CVDINFR4 = 1 THEN 1
            WHEN CVDINFR4 = 2 THEN 0
            WHEN CVDINFR4 IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [24]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS cvdstrk3_ever INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET cvdstrk3_ever =
        CASE
            WHEN CVDSTRK3 = 1 THEN 1
            WHEN CVDSTRK3 = 2 THEN 0
            WHEN CVDSTRK3 IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [25]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS chccopd1_ever INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET chccopd1_ever =
        CASE
            WHEN CHCCOPD1 = 1 THEN 1
            WHEN CHCCOPD1 = 2 THEN 0
            WHEN CHCCOPD1 IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [26]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS asthma3_ever INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET asthma3_ever =
        CASE
            WHEN ASTHMA3 = 1 THEN 1
            WHEN ASTHMA3 = 2 THEN 0
            WHEN ASTHMA3 IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [27]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS addepev2_ever INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET addepev2_ever =
        CASE
            WHEN ADDEPEV2 = 1 THEN 1
            WHEN ADDEPEV2 = 2 THEN 0
            WHEN ADDEPEV2 IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


In [28]:
conn.execute("""
    ALTER TABLE llcp2014_reduced
    ADD COLUMN IF NOT EXISTS diabetes_any INTEGER
""")

conn.execute("""
    UPDATE llcp2014_reduced
    SET diabetes_any =
        CASE
            WHEN DIABETE3 IN (1, 2) THEN 1
            WHEN DIABETE3 IN (3, 4) THEN 0
            WHEN DIABETE3 IN (7, 9) THEN NULL
            ELSE NULL
        END
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


Remove NaN sleep hours

In [29]:
conn.execute("""
    DELETE FROM llcp2014_reduced
    WHERE sleep_hours IS NULL
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_reduced").fetchone())


(458172,)


rename technical names for diseases

In [30]:
cols = {row[1].lower() for row in conn.execute("PRAGMA table_info('llcp2014_reduced')").fetchall()}

if "cvdinfr4_ever" in cols and "heartattack_ever" not in cols:
    conn.execute("ALTER TABLE llcp2014_reduced RENAME COLUMN cvdinfr4_ever TO heartattack_ever")

if "cvdstrk3_ever" in cols and "stroke_ever" not in cols:
    conn.execute("ALTER TABLE llcp2014_reduced RENAME COLUMN cvdstrk3_ever TO stroke_ever")

if "chccopd1_ever" in cols and "lungdisease_ever" not in cols:
    conn.execute("ALTER TABLE llcp2014_reduced RENAME COLUMN chccopd1_ever TO lungdisease_ever")

if "addepev2_ever" in cols and "depression_ever" not in cols:
    conn.execute("ALTER TABLE llcp2014_reduced RENAME COLUMN addepev2_ever TO depression_ever")


Establish final columns that will be used in modeling

In [31]:
conn.execute("""
    CREATE OR REPLACE TABLE llcp2014_clean AS
    SELECT
        sleep_hours,
        short_sleep,
        sex_male,
        _AGEG5YR,
        educ_3cat,
        low_income,
        race_white_nh,
        race_hispanic,
        any_leisure_pa,
        smoker_current,
        binge_drink,
        bmi,
        fairpoor_health,
        frequent_mental_distress,
        heartattack_ever,
        stroke_ever,
        lungdisease_ever,
        asthma3_ever,
        depression_ever,
        diabetes_any
    FROM llcp2014_reduced
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_clean").fetchone())


(458172,)


In [33]:
result = conn.execute("""
    SELECT column_name, 'numeric' AS feature_type
    FROM information_schema.columns
    WHERE table_name = 'llcp2014_clean'
      AND column_name IN ('bmi')

    UNION ALL

    SELECT column_name, 'ordinal' AS feature_type
    FROM information_schema.columns
    WHERE table_name = 'llcp2014_clean'
      AND column_name IN ('_AGEG5YR', 'educ_3cat', 'smoker_current')

    UNION ALL

    SELECT column_name, 'binary' AS feature_type
    FROM information_schema.columns
    WHERE table_name = 'llcp2014_clean'
      AND column_name IN (
        'sex_male', 'race_white_nh', 'race_hispanic',
        'any_leisure_pa', 'binge_drink', 'obese',
        'fairpoor_health', 'frequent_mental_distress',
        'heartattack_ever', 'stroke_ever', 'lungdisease_ever',
        'asthma3_ever', 'depression_ever', 'diabetes_any'
      )
""").fetchall()

print(result)


[('bmi', 'numeric'), ('_AGEG5YR', 'ordinal'), ('educ_3cat', 'ordinal'), ('smoker_current', 'ordinal'), ('sex_male', 'binary'), ('race_white_nh', 'binary'), ('race_hispanic', 'binary'), ('any_leisure_pa', 'binary'), ('binge_drink', 'binary'), ('fairpoor_health', 'binary'), ('frequent_mental_distress', 'binary'), ('heartattack_ever', 'binary'), ('stroke_ever', 'binary'), ('lungdisease_ever', 'binary'), ('asthma3_ever', 'binary'), ('depression_ever', 'binary'), ('diabetes_any', 'binary')]


In [42]:
feature_meta = conn.execute("""
    SELECT column_name, 'numeric' AS feature_type
    FROM information_schema.columns
    WHERE table_name = 'llcp2014_clean'
      AND column_name IN ('bmi')

    UNION ALL

    SELECT column_name, 'ordinal' AS feature_type
    FROM information_schema.columns
    WHERE table_name = 'llcp2014_clean'
      AND column_name IN ('_AGEG5YR', 'educ_3cat', 'smoker_current')

    UNION ALL

    SELECT column_name, 'binary' AS feature_type
    FROM information_schema.columns
    WHERE table_name = 'llcp2014_clean'
      AND column_name IN (
        'sex_male', 'race_white_nh', 'race_hispanic',
        'any_leisure_pa', 'binge_drink', 'obese',
        'fairpoor_health', 'frequent_mental_distress',
        'heartattack_ever', 'stroke_ever', 'lungdisease_ever',
        'asthma3_ever', 'depression_ever', 'diabetes_any'
      )
""").fetchall()

numeric_features = [name for name, ftype in feature_meta if ftype == "numeric"]
ordinal_features = [name for name, ftype in feature_meta if ftype == "ordinal"]
binary_features  = [name for name, ftype in feature_meta if ftype == "binary"]

## Data is clean  and ready to explore!

In [34]:
result = conn.execute("""
    SELECT 
        'bmi' AS feature,
        COUNT(*) FILTER (WHERE bmi IS NULL) AS num_missing,
        AVG(CASE WHEN bmi IS NULL THEN 1 ELSE 0 END) AS missing_ratio
    FROM llcp2014_clean
""").fetchall()

print(result)


[('bmi', 29548, 0.0644910644910645)]


In [35]:
result = conn.execute("""
    SELECT 
        feature,
        COUNT(*) FILTER (WHERE val IS NULL) AS num_missing,
        AVG(CASE WHEN val IS NULL THEN 1 ELSE 0 END) AS missing_ratio
    FROM (
        SELECT '_AGEG5YR' AS feature, _AGEG5YR AS val FROM llcp2014_clean
        UNION ALL
        SELECT 'educ_3cat', educ_3cat FROM llcp2014_clean
        UNION ALL
        SELECT 'smoker_current', smoker_current FROM llcp2014_clean
    ) t
    GROUP BY feature
    ORDER BY feature
""").fetchall()

print(result)


[('_AGEG5YR', 0, 0.0), ('educ_3cat', 3570, 0.007791833634530264), ('smoker_current', 20584, 0.044926359533101104)]


In [36]:
result = conn.execute("""
    SELECT 
        feature,
        COUNT(*) FILTER (WHERE val IS NULL) AS num_missing,
        AVG(CASE WHEN val IS NULL THEN 1 ELSE 0 END) AS missing_ratio
    FROM (
        SELECT 'sex_male' AS feature, sex_male AS val FROM llcp2014_clean 
        UNION ALL
        SELECT 'race_white_nh', race_white_nh FROM llcp2014_clean 
        UNION ALL
        SELECT 'race_hispanic', race_hispanic FROM llcp2014_clean 
        UNION ALL
        SELECT 'any_leisure_pa', any_leisure_pa FROM llcp2014_clean 
        UNION ALL
        SELECT 'binge_drink', binge_drink FROM llcp2014_clean 
        UNION ALL
        SELECT 'fairpoor_health', fairpoor_health FROM llcp2014_clean 
        UNION ALL
        SELECT 'frequent_mental_distress', frequent_mental_distress FROM llcp2014_clean 
        UNION ALL
        SELECT 'heartattack_ever', heartattack_ever FROM llcp2014_clean 
        UNION ALL
        SELECT 'stroke_ever', stroke_ever FROM llcp2014_clean 
        UNION ALL
        SELECT 'lungdisease_ever', lungdisease_ever FROM llcp2014_clean 
        UNION ALL
        SELECT 'asthma3_ever', asthma3_ever FROM llcp2014_clean 
        UNION ALL
        SELECT 'depression_ever', depression_ever FROM llcp2014_clean 
        UNION ALL
        SELECT 'diabetes_any', diabetes_any FROM llcp2014_clean
    ) t
    GROUP BY feature
    ORDER BY feature
""").fetchall()

print(result)


[('any_leisure_pa', 1628, 0.00355325074426198), ('asthma3_ever', 1414, 0.0030861772434806144), ('binge_drink', 28557, 0.06232812131688536), ('depression_ever', 1922, 0.004194931161223296), ('diabetes_any', 777, 0.0016958696733977634), ('fairpoor_health', 1515, 0.003306618475157801), ('frequent_mental_distress', 7211, 0.015738630907170235), ('heartattack_ever', 2155, 0.0047034738045974), ('lungdisease_ever', 2330, 0.00508542643374104), ('race_hispanic', 7542, 0.016461067022864775), ('race_white_nh', 7542, 0.016461067022864775), ('sex_male', 0, 0.0), ('stroke_ever', 1260, 0.0027500589298342106)]


In [37]:
conn.execute("""
    CREATE OR REPLACE TABLE llcp2014_clean_filtered AS
    SELECT *
    FROM llcp2014_clean
    WHERE bmi IS NOT NULL
      AND smoker_current IS NOT NULL
      AND binge_drink IS NOT NULL
""")

print(conn.execute("SELECT COUNT(*) FROM llcp2014_clean_filtered").fetchone())


(407198,)


In [38]:
result = conn.execute("SELECT * FROM llcp2014_clean_filtered").fetchdf()
df_filtered = result
print(df_filtered.shape)


(407198, 20)


In [39]:
duckdb.close()

### Modeling

In [40]:
target = "short_sleep"

print(df_filtered[target].value_counts())

df_model = df_filtered[df_filtered[target].notna()].copy()

X = df_model.drop(columns=[target])
y = df_model[target]


X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.25,
    stratify=y,
    random_state=42
)


short_sleep
0.0    278988
1.0    128210
Name: count, dtype: int64


In [43]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")), # give the numeric values a common value if missing
    ("scaler", StandardScaler()),
])

ordinal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), #give the ordinal values a common value if missing
    ("scaler", StandardScaler()),
])

binary_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), #give the binary values the most common value if missing 
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("ord", ordinal_transformer, ordinal_features),
        ("bin", binary_transformer, binary_features),
    ]
)


In [48]:
recall_scorer = make_scorer(recall_score, average="binary", pos_label=1)

param_grid = {
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__penalty": ["l2"],
    "clf__solver": ["liblinear", "lbfgs"], 
}

log_reg_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(
        max_iter=500,
        class_weight="balanced",
        n_jobs=-1
    )),
])

grid = GridSearchCV(
    estimator=log_reg_pipeline,
    param_grid=param_grid,
    scoring=recall_scorer,
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV recall:", grid.best_score_)

best_lr_pipeline = grid.best_estimator_
y_pred_lr = best_lr_pipeline.predict(X_val)
print(classification_report(y_val, y_pred_lr))
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred_lr))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs

Best params: {'clf__C': 0.01, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}
Best CV recall: 0.5359776333682331
              precision    recall  f1-score   support

         0.0       0.76      0.67      0.71     69747
         1.0       0.43      0.53      0.47     32053

    accuracy                           0.63    101800
   macro avg       0.59      0.60      0.59    101800
weighted avg       0.65      0.63      0.64    101800

Confusion matrix:
 [[46564 23183]
 [14916 17137]]


/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/iangower/Documents/GitHub/CS577/SemProjectTheSecond/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [52]:
rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=200,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1,
        verbose=1
    )),
])

param_grid = {
    "clf__max_depth": [None, 15],
    "clf__min_samples_split": [2, 10],
    "clf__min_samples_leaf": [1, 4],
    "clf__class_weight": [
        "balanced",
        {0: 1, 1: 3},
    ],
}

grid_rf = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid,
    scoring=recall_scorer,
    cv=3,
    n_jobs=-1,
)

grid_rf.fit(X_train, y_train)

print("Best params:", grid_rf.best_params_)
print("Best CV recall:", grid_rf.best_score_)

best_rf_pipeline = grid_rf.best_estimator_

y_pred_rf = best_rf_pipeline.predict(X_val)
print("Random Forest (best recall model)")
print(classification_report(y_val, y_pred_rf))
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred_rf))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs

Best params: {'clf__class_weight': {0: 1, 1: 3}, 'clf__max_depth': 15, 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 10}
Best CV recall: 0.7295360614247016
Random Forest (best recall model)
              precision    recall  f1-score   support

         0.0       0.79      0.45      0.58     69747
         1.0       0.38      0.74      0.50     32053

    accuracy                           0.54    101800
   macro avg       0.59      0.60      0.54    101800
weighted avg       0.66      0.54      0.55    101800

Confusion matrix:
 [[31728 38019]
 [ 8452 23601]]


In [55]:
rf_model = best_rf_pipeline.named_steps["clf"]

encoded_feature_names = best_rf_pipeline.named_steps["preprocess"].get_feature_names_out()

importances = rf_model.feature_importances_

feat_imp = (
    pd.DataFrame({"feature": encoded_feature_names, "importance": importances})
      .sort_values("importance", ascending=False)
      .reset_index(drop=True)
)

# 5. Display top features
feat_imp.head(20)


,feature,importance
0,num__bmi,0.217719
1,ord___AGEG5YR,0.181223
2,bin__frequent_mental_distress,0.100656
3,bin__fairpoor_health,0.088644
4,ord__smoker_current,0.078597
5,bin__race_white_nh,0.070981
6,ord__educ_3cat,0.066097
7,bin__any_leisure_pa,0.029944
8,bin__depression_ever,0.029639
9,bin__sex_male,0.025749
